In [2]:
set.seed(41)
library(dplyr)
library(ggplot2)
library(hBayesDM)
library(pheatmap)
library(truncnorm)
library(rstan)
library(cmdstanr)
library(bayesplot)
library(posterior)
library(loo)
library(rstanarm)
library(magick)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: Rcpp



This is hBayesDM version 1.2.1



Loading required package: StanHeaders


rstan version 2.32.7 (Stan version 2.32.2)


For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
For within-chain threading using `reduce_sum()` or `map_rect()` Stan functions,
change `threads_per_chain` option:
rstan_options(threads_per_chain = 1)


Do not specify '-march=native' in 'LOCAL_CPPFLAGS' or a Makevars file

This is cmdstanr version 0.9.0

- CmdStanR documentation and vignettes: mc-stan.org/cmdstanr

- CmdStan path: C:/Users/eb08/.cmdstan/cmdstan-2.36.0

- CmdStan version: 2.36.0

This is bayesplot 

In [3]:
#load draws
modelList <- c("1lr_stick0_blk3_allparamsep", "1lr_stick0_blk3_allparamshared", "1lr_stick0_blk3_emVsNot",
                "2lr_stick0_blk3_allparamsep", "2lr_stick0_blk3_allparamshared", "2lr_stick0_blk3_emVsNot",
                "1lr_stick1_blk3_allparamsep", "1lr_stick1_blk3_allparamshared", "1lr_stick1_blk3_emVsNot",
                "2lr_stick1_blk3_allparamsep", "2lr_stick1_blk3_allparamshared", "2lr_stick1_blk3_emVsNot")
rhat_neff_df <- data.frame()
for(modelName in modelList){
    filepath <- paste0("//cbsu/data/Group/Nord/DisgustReversalLearningModeling/finalModelComp/modelOutputs/", modelName, "/draws_", modelName, ".rds")
    draws <- readRDS(filepath)
    sumdraws <- summarise_draws(draws)
    max_rhat <- max(sumdraws$rhat)
    min_neff <- min(sumdraws$ess_bulk/dim(draws))

    temp_df <- data.frame(
                'modelName'=modelName,
                'max_rhat' =max_rhat,
                'min_neff'=min_neff
    )
    rhat_neff_df <- rbind(rhat_neff_df, temp_df)
}

In [4]:
rhat_neff_df <- rhat_neff_df[order(rhat_neff_df$min_neff), ] 
rhat_neff_df

,modelName,max_rhat,min_neff
,<chr>,<dbl>,<dbl>
11,2lr_stick1_blk3_allparamshared,1.013680,0.3090907
10,2lr_stick1_blk3_allparamsep,1.003421,0.3482681
4,2lr_stick0_blk3_allparamsep,1.005152,0.3515511
12,2lr_stick1_blk3_emVsNot,1.003234,0.3644348
6,2lr_stick0_blk3_emVsNot,1.004530,0.5568611
2,1lr_stick0_blk3_allparamshared,1.002824,0.6965478
5,2lr_stick0_blk3_allparamshared,1.003138,0.7574476
7,1lr_stick1_blk3_allparamsep,1.003552,0.7669077
9,1lr_stick1_blk3_emVsNot,1.003008,0.8099417


In [5]:
rhat_neff_df <- rhat_neff_df[order(rhat_neff_df$max_rhat), ] 
rhat_neff_df

,modelName,max_rhat,min_neff
,<chr>,<dbl>,<dbl>
2,1lr_stick0_blk3_allparamshared,1.002824,0.6965478
3,1lr_stick0_blk3_emVsNot,1.003005,0.8257590
9,1lr_stick1_blk3_emVsNot,1.003008,0.8099417
5,2lr_stick0_blk3_allparamshared,1.003138,0.7574476
12,2lr_stick1_blk3_emVsNot,1.003234,0.3644348
1,1lr_stick0_blk3_allparamsep,1.003318,0.8133144
10,2lr_stick1_blk3_allparamsep,1.003421,0.3482681
7,1lr_stick1_blk3_allparamsep,1.003552,0.7669077
8,1lr_stick1_blk3_allparamshared,1.004014,0.8345388


In [9]:
write.csv(rhat_neff_df, 'rhat_neff_df.csv', row.names=FALSE)